In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# input
data_d <- '/oak/stanford/groups/mrivas/projects/biobank-methods-dev/snpnet-elastic-net'
phe_f  <- file.path(data_d, 'phenotype.phe')

# constants
covars <- c('age', 'sex', paste0('PC', 1:10))
alphas <- c(0, 0.1, 0.5, 0.9)

# output
out_f <- 'snpnet-elastic-net.eval.tsv'


In [3]:
read_PRS <- function(GBE_ID, alpha, data_dir=data_d){
    sscore_f <- file.path(
        data_dir,
        sprintf('%s_%s', GBE_ID, alpha),
        sprintf('%s.sscore.zst', GBE_ID)
    )
    fread(
        cmd=paste('zstdcat', sscore_f),
        select=c('#FID', 'IID', 'SCORE1_SUM'),
        colClasses=c('#FID'='character', 'IID'='character')
    ) %>%
    rename('FID'='#FID', 'geno_score'='SCORE1_SUM')
}


In [4]:
read_covars <- function(GBE_ID, alpha, data_dir=data_d){
    file.path(
        data_dir,
        sprintf('%s_%s', GBE_ID, alpha),
        'snpnet.covars.tsv'
    ) %>%
    fread(colClasses=c('ID'='character')) %>%
    column_to_rownames('ID')
}


In [5]:
read_BETAs <- function(GBE_ID, alpha, data_dir=data_d){
    file.path(
        data_dir,
        sprintf('%s_%s', GBE_ID, alpha),
        'snpnet.tsv'
    ) %>%
    fread(colClasses=c('ID'='character'))
}


In [6]:
read_predicted_scores <- function(phe_df, GBE_ID, alpha, covariates=covars){
    covar_df <- read_covars(GBE_ID, alpha)
    as.matrix(
        phe_df %>% select(all_of(covariates))
    ) %*% as.matrix(covar_df) %>%
    as.data.frame() %>%
    rownames_to_column('ID') %>%
    separate(ID, c('FID', 'IID')) %>% 
    rename('covar_score'='BETA') %>%
    left_join(
        phe_df %>% select(FID, IID, split, all_of(GBE_ID)),
        by=c('FID', 'IID')
    ) %>%
    left_join(
        read_PRS(GBE_ID, alpha),
        by=c('FID', 'IID')
    ) %>%
    mutate(
        geno_covar_score = geno_score + covar_score
    )
}


In [7]:
perform_eval <- function(response, pred, metric.type){
    if(metric.type == 'r2'){
        summary(lm(response ~ 1 + pred))$r.squared
    }else{
#         pROC::auc(pROC::roc(response, pred))        
        pred.obj <- ROCR::prediction(pred, factor(response - 1))
        auc.obj <- ROCR::performance(pred.obj, measure = 'auc')
        auc.obj@y.values[[1]]
    }
}


In [8]:
build_eval_df <- function(phe_df, GBE_ID, alpha, split_string, metric.type){
    score_test_df <- phe_df %>%
    read_predicted_scores(GBE_ID, alpha) %>%
    filter(split == split_string) %>%
    drop_na(all_of(GBE_ID)) %>%
    filter(GBE_ID != -9)

    data.frame(
        GBE_ID     = GBE_ID,
        alpha      = alpha,
        n_variables = read_BETAs(GBE_ID, alpha) %>% nrow(),
        geno       = perform_eval(
            score_test_df[[GBE_ID]],
            score_test_df$geno_score,
            metric.type
        ),
        covar      = perform_eval(
            score_test_df[[GBE_ID]],
            score_test_df$covar_score,
            metric.type
        ),
        geno_covar = perform_eval(
            score_test_df[[GBE_ID]],
            score_test_df$geno_covar_score,
            metric.type
        ),
        stringsAsFactors = F
    )    
}


In [9]:
get_eval_df <- function(phe_df, alphas, split_string){
    eval_df <- bind_rows(
        alphas %>% lapply(function(alpha){ tryCatch({ 
            phe_df %>% build_eval_df('INI50', alpha, split_string, 'r2')
        }, error=function(e){})}) %>% bind_rows(),

        alphas %>% lapply(function(alpha){ tryCatch({ 
            phe_df %>% build_eval_df('INI21001', alpha, split_string, 'r2')
        }, error=function(e){})}) %>% bind_rows(),

        alphas %>% lapply(function(alpha){ tryCatch({ 
            phe_df %>% build_eval_df('HC269', alpha, split_string, 'auc')
        }, error=function(e){})}) %>% bind_rows(),

        alphas %>% lapply(function(alpha){ tryCatch({ 
            phe_df %>% build_eval_df('HC382', alpha, split_string, 'auc')
        }, error=function(e){})}) %>% bind_rows()
    )
}


## compute the performance metric

In [10]:
phe_df <- fread(phe_f, colClasses=c('FID'='character', 'IID'='character')) %>%
mutate(ID = paste(FID, IID, sep='_')) %>%
column_to_rownames('ID')


In [11]:
eval_df <- bind_rows(
    phe_df %>% get_eval_df(alphas, 'train') %>% mutate(split = 'train'),
    phe_df %>% get_eval_df(alphas, 'val')   %>% mutate(split = 'val'),
    phe_df %>% get_eval_df(alphas, 'test')  %>% mutate(split = 'test')
)


In [13]:
eval_df %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


## visualization

In [14]:
trait_names <- data.frame(
    trait  = c('Standing height', 'Body mass index', 'High cholesterol', 'Asthma'),
    GBE_ID = c('INI50', 'INI21001', 'HC269', 'HC382'),
    stringsAsFactors=F
)


In [15]:
p_pred <- eval_df %>%
filter(split == 'test') %>%
left_join(trait_names, by='GBE_ID') %>%
mutate(label = sprintf('%.4f\n(%d)', geno_covar, n_variables))%>%
ggplot(aes(x=as.factor(alpha), y=geno_covar, color=as.factor(alpha), fill=as.factor(alpha))) +
geom_bar(stat="identity") + 
theme_bw() +
geom_text(aes(label=label), vjust=1.2, color="white", size=3.5)+
theme(legend.position='none')+
labs(
    x = 'alpha parameter in elastic net',
    y = 'Predictive performance (w/ genetics + covariates) on the test set (R2 or AUC)',
    title = 'Predictive performance and the number of genetic features in snpnet-elasticnet'
)+
# geom_text(aes(label=round(geno_covar, 4)), vjust=1.6, color="white", size=3.5)+
facet_wrap(~trait)


In [16]:
p_size <- eval_df %>%
filter(split == 'test') %>%
left_join(trait_names, by='GBE_ID') %>%
mutate(label = sprintf('%d\n(%.4f)', n_variables, geno_covar))%>%
ggplot(aes(x=as.factor(alpha), y=n_variables, color=as.factor(alpha), fill=as.factor(alpha))) +
geom_bar(stat="identity") + 
theme_bw() +
geom_text(aes(label=label), vjust=0.65, color="black", size=3.5)+
theme(legend.position='none')+
labs(
    x = 'alpha parameter in elastic net',
    y = 'The number of non-zero BETAs in genetic features',
    title = 'Predictive performance and the number of genetic features in snpnet-elasticnet'
)+
# geom_text(aes(label=round(geno_covar, 4)), vjust=1.6, color="white", size=3.5)+
facet_wrap(~trait)


In [17]:
ggsave(
    '20200527_elasticnet-ridge-plot.1.png', p_pred
)

ggsave(
    '20200527_elasticnet-ridge-plot.2.png', p_size
)


Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
